In [5]:
import pandas as pd
from IPython.display import display
from pyvis.network import Network

from db_inference import calc_icity
from db_inference.simple_sql_db import SimpleSqlDb
from utils import notebook_util

notebook_util.disp_notebook_full_width()


In [6]:
sql_db = SimpleSqlDb()

In [3]:
# Interesting combo (high scoring)
# bait_hash = "693388b375dbd274e1"
# tgt_hash = "1b58be4f65b975319f"

bait_hash = "693388b375dbd274e1"
print("BAIT", bait_hash)
tgt_hashes = sql_db.get_p100_windowed_neighbors(bait_hash)
print("TARGETS")
display(tgt_hashes)

tgt_hash = tgt_hashes[0]

BAIT 693388b375dbd274e1
TARGETS


['1b58be4f65b975319f',
 '59f3653c36dde18eef',
 '6c403b38891422234d',
 '7426e51f0050136f07',
 '8282ec0833b44af2e8',
 'b078cbc8fc25b6bf55',
 'b4a0ae3ee9d63bb120',
 'c25c022677394b4a2f',
 'ce3ef3eefef3823f5a',
 'd7dc8eb56b5745f9bc',
 'db310e2a5678bbc1cf',
 'e1912a410e1beb69fe',
 'f3fbe07a74adf91ccc']

In [ ]:
%%time
bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']

tgt_results = []

for tgt_hash in tgt_hashes:
    tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
    tgt_res, bait_res, G = calc_icity.calc_icity_tgt_bait(sql_db, tgt_p30, bait_p30)
    tgt_results.append(tgt_res)
    print(f"calced {tgt_hash}")

icity_df = pd.DataFrame(tgt_results)
icity_df.sort_values('icity_p100', ascending=False, inplace=True)
display(icity_df)

In [8]:
bait_p30 = "557e309d5f3d13e440"
tgt_p30 = "3c157c75f39f8ab820"

# tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
# bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']
tgt_res, bait_res, G = calc_icity.calc_icity_tgt_bait(sql_db, tgt_p30, bait_p30)
tgt_res

{'tgt_hash': '3c157c75f39f8ab820',
 'num_tgt_p100s': 689,
 'num_tgt_p100s_connected': 544,
 'icity_p100': 0.7895500725689405,
 'num_tgt_p90s': 226,
 'num_tgt_p90s_any_p100_connected': 164,
 'num_tgt_p90s_majority_p100_connected': 160,
 'icity_p90_any': 0.7256637168141593,
 'icity p90 majority': 0.7079646017699115}

In [ ]:
%%time
# Graph connections between target cluster (blue) and bait cluster (purple)
tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']
G = calc_icity.build_icity_graph(sql_db, tgt_p30, bait_p30)

In [ ]:
%%time
net = Network(notebook=True, height="1000px", width="1000px")
net.from_nx(G)
net.show('./pyvis_test.html')